# Table of Contents
1. [notes](#notes)
2. [start](#start)
3. [pwd](#pwd)
4. [dns-ldap](#dns-ldap)
5. [recon](#recon)
6. [fuzz](#fuzz)
7. [print](#print)

# Notes <a name="notes"></a>
## Tools
### CyberChef
```
https://gchq.github.io/CyberChef/
```
### Javascript Deobfuscate
```
docker run -d --rm --name jsdetox -p 3000:3000 remnux/jsdetox
docker run -d --rm -p 4000:4000 -p 35729:35729 --name de4js remnux/de4js
```
### JWT
```
docker run -it --network "host" --rm -v "${PWD}:/tmp" -v "${HOME}/.jwt_tool:/root/.jwt_tool" ticarpi/jwt_tool
```
### Podman vs docker
```
Just set podman as docker, then run normal docker commands.
Since so many scripts rely on docker, just go ahead and map this over so it's not an issue, and podman can manage these
alias docker='podman' 
```

## LibVirt/Virtualization
### Start local python server (to pull isos, instead of from other location)
```
python3 -m http.server --directory=/share/vms/isos/
```
### Create VM
```
papermill /share/public-git-repos/automation-tools/libvirt-provision.ipynb /dev/stdout -p hostname ubuntu -p domainname private -p url http://10.42.0.1:8000/noble-server-cloudimg-amd64.img -p os_variant ubuntu-stable-latest -p size 40G -p ram 2048
```
### Power on all stopped vms
```
for i in `virsh list --all|grep -i off|awk '{print $2}'`; do virsh start $i; done
```
### Stop and remove all vms
```
for i in `virsh list --all|grep -i .private|awk '{print $2}'`; do tmux new-session -d "virsh destroy $i; sleep 300; virsh undefine $i --remove-all-storage"; done 2>/dev/null
```
### Get dhcp lease information of vms
```
virsh net-dhcp-leases default
```
### uptime check
```
nmap -sS -p22 --open -Pn -n `ip address show dev $(virsh net-info default|grep -i bridge |awk '{print $2}')|grep -i inet|awk '{print $2}'`|grep -i "report for" |awk '{print $5}' > /tmp/virtnmap; for i in `cat /tmp/virtnmap`; do ssh-keyscan -H $i > ~/.ssh/known_hosts 2>/dev/null; ansible $i -i /tmp/virtnmap -a "uptime" -T 30 -u ansible -b; done
```


## Begin <a name="start"></a>

In [ ]:
# expected to run with papermill, so gonna do some additional santizing as i can
input_data="10.13.37.15"

In [ ]:
#papermill_description=Setup_target

import netaddr, netifaces, pwn
from dns import resolver
iplist = [netifaces.ifaddresses(i)[netifaces.AF_INET][0]["addr"] for i in netifaces.interfaces() if netifaces.AF_INET in netifaces.ifaddresses(i)]
print("Local Interfaces: ")
for i in iplist: 
    print(i)
ip=""
Host=""
input_data=str(input_data)
print("Target: "+input_data)
try:
    try:
        # try to identify if this is a valid ip address
#        print(netaddr.valid_ipv4(input_data))
        if netaddr.valid_ipv4(input_data):
            ip=input_data
        elif netaddr.valid_ipv6(input_data):
            ip=input_data
        else:
            ip=""
#        print(ip)
    except:
        print('not an ip address')
    if ip == "":
        # this case exists to resolve the domain name to provide both ip and host
        try:
            ip=resolver.resolve(input_data, "A").response.answer[0][0].address
            print('ip = '+str(ip))
            Host=str(input_data)
        except:
            print('host lookup failed')
            # this would mean ip = "" and the input data isn't a resolvable name
    else:
        #this case is if ip exists, set host to ip address
        Host=str(input_data)
except:
    print("put proper address")
    exit()
    

## Setup working directory <a name="pwd"></a>

In [ ]:
#papermill_description=SetupWorkingDirectory
import re, requests, ldap3
hostsString=ip+" "+Host
!echo {hostsString} >> /etc/hosts
workingDir="/share/pentests/"+str(Host)
!mkdir {workingDir}
%cd {workingDir}

## Check if either DNS or LDAP <a name="dns-ldap"></a>

In [ ]:
#papermill_description=CheckDNS_and_LDAP
try:
    if Host != ip:
        testdns=resolver.Resolver(configure=False)
        testdns.nameservers=[ip]
        testdns.resolve(Host, "a")
    !dnsrecon -r {ip}/32 -n {ip} > self-dns-check
    !dnsrecon -r {ip}/28 -n {ip} >> self-dns-check
    !dnsrecon -r {ip}/24 -n {ip} >> self-dns-check
    !dnsrecon -r 127.0.0.1/24 -n {ip} >> self-dns-check
    !dnsrecon -d {Host} -a >> self-dns-check
    !dnsrecon -d {Host} -D /share/pentests/wordlists/dns.txt >> self-dns-check
    command="""n=`awk -F "\t" '{print $2}' self-dns-check |sort -u|grep -iv localhost|sed /^[[:space:]]*$/d|awk '{print $2}' |sed s/\$/" "/g|tr -d '\\n'`; o=`grep -m1 """+Host+""" /etc/hosts`; e=`whatweb http://"""+Host+""" 2>&1|grep -i error|awk '{print $8}'|strings`; sed "s/$o/$o $n $e/g" -i /etc/hosts"""
    !{command}
except:
    print('not a dns server')
    pass
try:
 server=ldap3.Server(ip, get_info=ldap3.ALL, port=389, use_ssl=False)
 connection=ldap3.Connection(server)
 connection.bind()
 dcinfo=server.info
 print(dcinfo)
except:
    print('likely not an ldap server')
    pass

## Start recon <a name="recon"></a>

In [ ]:
#papermill_description=Nmap_and_whatweb
!nmap -sS -sV -p1- -Pn -n --open -oA {Host}.nmap {ip}
result=!grep -h http {Host}.nmap.nmap|grep -i "open"|awk -F "/" '{{print $$1}}'
print(str(result))
!cat {Host}.nmap.xml |jc --xml|jq . > {Host}-nmap.json
for port in result:
    !whatweb http://{Host}:$port >> whatweb-{Host}
    !whatweb https://{Host}:$port >> whatweb-{Host}

## Start Fuzz <a name="fuzz"></a>

In [ ]:
#papermill_description=Begin_Fuzzing
def fuzzing():
    for port in result:
        for line in open("/etc/hosts","r").readlines():
            if Host in line:
                for name in set(str(line).replace("\n","").split(" ")):
                    if "." in name:
                        #print(name+":"+port)
                        try:
                            testname='.'.join(name.split('.')[-2:])
                            check=requests.get('http://'+name+":"+port, timeout=20)
                            length=str(len(check.content))
                            if check.status_code: #check if http
                                command="ffuf -s -v -u http://"+name+":"+port+"/FUZZ -w /share/pentests/wordlists/webcontent-top-10000.txt -o ffuf-"+name+"-"+port+"-http-log >/dev/null"
                                output=!{command}
                                #print(output)
                                command="ffuf -s -v -u http://"+name+":"+port+" -H 'Host: FUZZ."+name+"' -w /share/pentests/wordlists/dns-top-10000.txt -o ffuf-"+name+"-"+port+"-dns-log >/dev/null"
                                output=!{command}
                                #print(output)
                                if name != ip: #instead of re-checking that it's an ip, just check to make sure an ip wasn't pulled from hosts (a situation that happens when overwriting it wrong
                                    command="ffuf -s -v -u http://"+name+":"+port+" -H 'Host: FUZZ."+testname+"' -w /share/pentests/wordlists/dns-top-10000.txt -fs "+length+" -o ffuf-"+testname+"-"+port+"-dns-log >/dev/null"
                                    output=!{command}
                                    #print(output)
                            else: #just go ahead and try https, if this fails, it's irrelevant to put into another if clause
                                command="ffuf -s -v -u https://"+name+":"+port+"/FUZZ -w /share/pentests/wordlists/webcontent-top-10000.txt -o ffuf-"+name+"-"+port+"-http-log >/dev/null"
                                output=!{command}
                                #print(output)
                                command="ffuf -s -v -u https://"+name+":"+port+" -H 'Host: FUZZ."+name+"' -w /share/pentests/wordlists/dns-top-10000.txt -o ffuf-"+name+"-"+port+"-dns-log >/dev/null"
                                output=!{command}
                                #print(output)
                                if name != ip: #instead of re-checking that it's an ip, just check to make sure an ip wasn't pulled from hosts (a situation that happens when overwriting it wrong
                                    command="ffuf -s -v -u https://"+name+":"+port+" -H 'Host: FUZZ."+testname+"' -w /share/pentests/wordlists/dns-top-10000.txt -fs "+length+" -o ffuf-"+testname+"-"+port+"-dns-log >/dev/null"
                                    output=!{command}
                                    #print(output)
                        except Exception as e: 
                            #print(e)
                            pass
                break
fuzzing()
command="""n=`awk -F "\t" '{print $2}' self-dns-check |sort -u|grep -iv localhost|sed /^[[:space:]]*$/d|awk '{print $2}' |sed s/"$"/" "/g|tr -d '\\n'`; o=`grep -m1 """+Host+""" /etc/hosts`; e=`cat ffuf-*-dns-log|jq -c '.results[] | {url:.url,host:.host,status:.status,length:.length}|map(.)'|awk -F ',' '!s[$4]++' |awk -F '"' '{print $4}'|sort|uniq |sed s/"$"/" "/g|tr -d '\\n'`; sed "s/$o/$o $n $e/g" -i /etc/hosts; grep """+Host+""" /etc/hosts"""
output=!{command}
data=[]
for i in output:
    for o in i.split(' '):
        data.append(o)
print(set(data))
fuzzing()
command="""for each in `ls ffuf-*`; do cat $each | jq -c '.results[] | {url:.url,status: .status}'; done|sort -u"""
output=!{command}
for i in output:
    if "200" in i or "300" in i or "302" in i or "503" in i or "302" in i:
        print(i)

## Use fuzz results to crawl <a name="print"></a>

In [ ]:
#papermill_description=CrawlUrlList
!cat ffuf-*-log |jq ".results[] |.url, .input.FUZZ, .status"|sed 'N;N;s/\n/ /g' |sed s/'" "'/'\/'/g|sed s/'"'//g|grep -i "200\|301" |awk '{print $1}'|sort -u |docker exec -i kali-0 hakrawler |sort -u -T ./